In [72]:
# Reproduce book environment
import random
random.seed(1234)

import logging
from pprint import pprint
from sys import stdout as STDOUT

# Write all output to a temporary directory
import atexit
import gc
import io
import os
import tempfile

# 6章 メタクラスと属性

メタクラスは、Python の特長として挙げられる概念の一つである。

メタクラスという名前は漠然とクラスより上の概念を意味している。単純に言えば、メタクラスは Python の `class` 文に割り込み、クラスが定義されるたびに特別な振る舞いを与えるものである。

同様の不思議さと力を持っているのが、Python に組み込まれている属性アクセスを動的にカスタマイズする機能である。それらは、Python のオブジェクト指向機能とともに、単純なクラスから複雑なクラスへと移行する際に威力を発揮する。

しかしながら、これらの強力な機能には、落とし穴が多くある。例えば、動的属性というものを使用する際には、オブジェクトをオーバーライドして、予期しない副作用を起こすことがある。メタクラスは、初心者には手に負えない、とても奇怪な振る舞いが生じることが多い。驚き最小の原則（[Principle of least astonishment / Rule of least surprise](https://ja.wikipedia.org/wiki/%E9%A9%9A%E3%81%8D%E6%9C%80%E5%B0%8F%E3%81%AE%E5%8E%9F%E5%89%87)）に従い、これらの機構をよく理解されたイディオムを実装するためだけに使うことが重要である。

## 復習 アンダースコア_付きクラス変数 / インスタンス変数

### アンダースコア_1個のクラス変数 / インスタンス変数

アンダースコア1個で始まるクラス変数やインスタンス変数は、慣習的に「隠蔽したい」ことを表している。

これはあくまで「参照しないで欲しい」という意思表示であり、実際には自クラス以外からも参照できてしまう。

しかし、このような変数は自クラス以外から参照しないようにすべきである。

In [73]:
class A:
    def __init__(self, x):
        self._x = x

a = A(10)
a._x    # 参照できてしまう

10

### アンダースコア_2個のクラス変数 / インスタンス変数

アンダースコア2個で始まるクラス変数やインスタンス変数は、自クラス以外から隠蔽される。
しかし実際には、名前マングリング（名前難号化）という機能により変数名が変更されるだけである。
そのため、変更された変数名を使えば自クラス以外からも参照できてしまう。

たとえば、`Sample` クラスの `__num3` という変数は、下記のように `_Sample__num3` に変わる。
ただし、デバッグなどの特殊な状況以外では、`_Sample__num3` を使って直接アクセスするのは控えるべきである。

In [74]:
class Sample:
    def __init__(self):
        self.__num3 = 300


a = Sample()
print(a._Sample__num3)  # 変数名が変更されているが参照できる

300


参考サイト：[アンダースコア（_）ではじまるクラスの変数](https://blog.pyq.jp/entry/python_kaiketsu_220323)

## 項目44 `get` メソッドや `set` メソッドは使わず属性をそのまま使う

他の言語から Python に移ってきたプログラマは、クラスのゲッター・セッターメソッドを明示的に実装しようとしたくなることがあるだろう。

例えば抵抗器（Resistor）クラスが抵抗度合いのオーム（ohrms）をインスタンス変数に持つとしよう。`get_ohms` メソッドではその `ohrms` の値を返し、`set_ohms` メソッドではその `ohrms` の値を変更するように実装する（これらをゲッター・セッターメソッドと呼んでいる）。

In [75]:
# Example 1
class OldResistor:
    def __init__(self, ohms):
        self._ohms = ohms

    def get_ohms(self):
        return self._ohms

    def set_ohms(self, ohms):
        self._ohms = ohms

このゲッターやセッターの実装の仕方や、次のコードのような使い方は単純ではあるが、Pythonic では無い。

※ コードがpythonicであるというのは、
- Pythonのイディオムをうまく使っていること
- 自然であること
- （言語に）流暢であること
- Pythonの最小主義の哲学と読みやすさの強調に適合していること

を意味する。The Zen of Pythonについては以前の田保資料参照に記載（[他参考記事](https://www.pythonic-exam.com/pythonic)）。

In [76]:
# Example 2
r0 = OldResistor(50e3)
print('Before:', r0.get_ohms())
r0.set_ohms(10e3)
print('After: ', r0.get_ohms())

Before: 50000.0
After:  10000.0


特に、次のような加減演算などを伴う場合では、ぎこちないコードになってしまっている。

In [77]:
# Example 3
r0.set_ohms(r0.get_ohms() - 4e3)
assert r0.get_ohms() == 6e3

こういったユーティリティメソッド※の実装はクラスのインタフェース定義を助け、機能をカプセル化して使い方を実証し、境界を定義するのを容易にするものである。これらは、クラスが時間が経つとともに進化しても（機能拡張を行っても？）呼び出し元に問題が起こらないことを保証する、クラスを設計する際に重要な目標である。

※ユーティリティ：「役に立つこと」「有益なもの」（[参考:weblio辞書](https://www.weblio.jp/content/%E3%83%A6%E3%83%BC%E3%83%86%E3%82%A3%E3%83%AA%E3%83%86%E3%82%A3%E3%83%BC)）

しかし実は、Python では、明示的にセッターやゲッターメソッドを実装する必要はほとんど無い。例えば、次のように単純なパブリック属性を用いて実装することができるからである。

In [78]:
# Example 4
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

r1 = Resistor(50e3)
r1.ohms = 10e3  # set メソッド
print(f'{r1.ohms} ohms, '
      f'{r1.voltage} volts, '
      f'{r1.current} amps') # get メソッド

10000.0 ohms, 0 volts, 0 amps


このように書くことで、増加演算なども自然で明確になる。

In [79]:
# Example 5
r1.ohms += 5e3

# 比較：Example 3
# r0.set_ohms(r0.get_ohms() - 4e3)

後になって、属性が設定されたときに特別な振る舞いが必要となる場合は、※`@property` デコレータ（背景は「項目 26 `functools.wraps` を使って関数デコレータを定義する」参照）とそれに対応する `setter` 属性をマイグレートすれば良い。

#### 捕捉：`@property` について

プロパティとは、値を変更しづらくしたいが、インスタンス変数のように自然に値にアクセスできるようにできるよう設計されたもの。（[参考](https://qiita.com/cardene/items/8a59d576d360b7568c3a)）

In [80]:
'プロパティの値を取り出すメソッドを定義する'
@property
def プロパティ名(self):
  return 値
  
'プロパティの値を設定'
@プロパティ名.setter
def プロパティ名(self, 値):
  pass  # 値を処理するコード

In [81]:
# 参考
class PropertyClass:
      def __init__(self, msg):
        self.message = msg
  
      def __str__(self):
        return self.message
    
      @property
      def message(self):
        return self.__message
    
      @message.setter
      def message(self, value):
        if value != '':
          self.__message = value
      

pc= PropertyClass('Hello')
print(pc.message)
pc.message = ''   # messageの中身を「Hello」から空文字列に変更   ③
print(pc)   # Hello
pc.message = 'Bye!'   # messageの中身を「Hello」から「Bye!」に変更   ⑤
print(pc)   # Bye!

Hello
Hello
Bye!


細かい挙動は下記のコードで確認できる

In [82]:
class Myclass:
    def __init__(self, input_name):
        self.hidden_name = input_name

    @property
    def name(self):
        print('inside the getter')
        return self.hidden_name

    @name.setter
    def name(self, input_name):
        print('inseide the setter')
        self.hidden_name = input_name


mc = Myclass('Hello')   #「Hello」を引数として渡す
print(mc.name)   #'Hello'と表示 + 'inside the getter'と表示
mc.name = 'Python'   # 値を'Python'に変更 + 'inside the setter'と表示
print(mc.name)   # 'Python'と表示 + 'inside the getter'と表示

inside the getter
Hello
inseide the setter
inside the getter
Python


#### 以下書籍部分に戻る

次のコードでは、プロパティの `voltage` に値を代入することで、`current` を変更できる `Resistor` の新たなサブクラス `VoltageResistance` を定義する。これを正しく動作させるためには、セッターメソッドとゲッターメソッドの両方の名前が、意図している属性名に合致していなければならない。


In [83]:
# Example 4
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        

# Example 6
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0

    @property
    def voltage(self):
        return self._voltage

    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

`voltage` に値を代入すると、`voltage` のセッターメソッドが実行されて、対応するオブジェクトの `current` 属性が更新される。

In [84]:
# Example 7
r2 = VoltageResistance(1e3)
print(f'Before: {r2.current:.2f} amps')
r2.voltage = 10
print(f'After:  {r2.current:.2f} amps')

Before: 0.00 amps
After:  0.01 amps


### property の setter

プロパティの `setter` を指定することで、クラスに渡される値について型や値について精査することができる。

例えば、抵抗値が0オームよりも大きい値であることを確かめるクラスを次のように定義することができる：


In [85]:
# Example 4
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        

# Example 8
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'ohms must be > 0; got {ohms}')
        self._ohms = ohms

正しくない抵抗値（例えば0）を `ohms` に代入しようとすると例外が発生する。

In [86]:
# Example 9
try:
    r3 = BoundedResistance(1e3)
    r3.ohms = 0
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_15976\2725580157.py", line 4, in <cell line: 2>
    r3.ohms = 0
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_15976\586676205.py", line 21, in ohms
    raise ValueError(f'ohms must be > 0; got {ohms}')
ValueError: ohms must be > 0; got 0


同様に、コンストラクタに0未満の値を渡しても例外が発生する。

In [87]:
# Example 10
try:
    BoundedResistance(-5)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_15976\4192645586.py", line 3, in <cell line: 2>
    BoundedResistance(-5)
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_15976\586676205.py", line 12, in __init__
    super().__init__(ohms)
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_15976\586676205.py", line 4, in __init__
    self.ohms = ohms
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_15976\586676205.py", line 21, in ohms
    raise ValueError(f'ohms must be > 0; got {ohms}')
ValueError: ohms must be > 0; got -5


ここでは、
1. `BoundedResistance.__init__` が `Resistor.__init__` を呼び出し、その `self.ohms = -5` という代入が行われる
2. この代入により、`BoundedResistance` の `@ohms.setter` メソッドが呼び出され、オブジェクトの構築が完了する前に`@ohms.setter` メソッド内の `if` 文による検証が行われエラーが吐かれる。

In [88]:
# Example 4
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        

# Example 8
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'ohms must be > 0; got {ohms}')
        self._ohms = ohms

`@property` を使って、スーパークラスの属性を変更不能にすることすらできる。

In [89]:
# Example 11
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):  # hasattr により self が _ohms 属性を持っているか検証
            raise AttributeError("Ohms is immutable")
        self._ohms = ohms

構築後にプロパティへ値を代入しようとすると、例外が発生する。

In [90]:
# Example 12
try:
    r4 = FixedResistance(1e3)
    r4.ohms = 2e3
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_15976\2866752537.py", line 4, in <cell line: 2>
    r4.ohms = 2e3
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_15976\3329566724.py", line 13, in ohms
    raise AttributeError("Ohms is immutable")
AttributeError: Ohms is immutable


`@property` メソッドを使って、セッターやゲッターを実装するとき、予想しないような振る舞いを実装してはならない。

例えば、次のようにゲッタープロパティメソッドの中で、他の属性をセットさせるケースを見てみる：

In [91]:
# Example 13
class MysteriousResistor(Resistor):
    @property   # getter
    def ohms(self):
        self.voltage = self._ohms * self.current    # 他属性
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms

これは、ひどく異様な振る舞いを引き起こす。

In [92]:
# Example 14
r7 = MysteriousResistor(10)
r7.current = 0.01
print(f'Before: {r7.voltage:.2f}')
r7.ohms
print(f'After:  {r7.voltage:.2f}')


Before: 0.00
After:  0.10


`@property.setter` メソッドでは、関連するオブジェクト状態だけを変更することが最良だろう。

- オブジェクトを超えて、モジュールを動的にインポートする
- 遅いヘルパー関数を実行する
- 入出力を行う
- 高くつくデータベースクエリを行う

といった、呼び出し元が予期しない副作用をもたらしていないことを確かめて実装すべきである。

クラスのユーザというものは、その属性が Python の他のオブジェクトと同じように、さっと使いやすいものと期待している。複雑であったり、遅くなるようなことは、通常のメソッドを使って行うべきである。


`@property` の最大の欠点は、属性のメソッドがサブクラスの間でしか共有できないことである。関連しないクラスは、同じ実装を共有することができない。しかし、Python では、ディスクリプタもサポート（「項目46 再利用可能な `@property` メソッドにディスクリプタを使う」参照）しており、プロパティの再利用可能なロジックや他の多くのユースケースを可能にしている。

### 覚えておくこと

- 単純なパブリックな属性を使って新たなクラスのインタフェースを定義し、`set` や `get` メソッドは定義しない。
- 必要ならオブジェクトの属性にアクセスされたときの特別な振る舞いを `@property` を使っ
て定義する。
- 驚き最小の原則に従い、`@property` メソッドで奇妙な副作用が生じるのを防ぐ。


## 項目45 属性をリファクタリングする代わりに `@property` を考える

組み込みの `@property` デコレータによって、 インスタンスの属性への単純なアクセスが容易にできる（項目44 参照）。

よく使われる `@property` の高度な使い方の一つは、単純な数値属性を、その場で計算するように変えるものである。これは既存のクラス利用のすべてを、呼び出しを一切変えることなく、新たな振る舞いができるようにマイグレートするため、非常に役立つ。時間が経つにつれ変化するインタフェースを改善していくときの重要な応急処置にもなる。

例えば、普通の Python オブジェクトを用いて水漏れバケツからの水の割り当てを実装することを考える。ここで、`Bucket` クラスは、残っている割当量と、割当量の存在する時間（ピリオド）を属性に持つよう設計したとする：


In [93]:
# Example 1
from datetime import datetime, timedelta

class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0

    def __repr__(self):
        return f'Bucket(quota={self.quota})'

bucket = Bucket(60)
print(bucket)


Bucket(quota=0)


水漏れバケツアルゴリズムでは、バケツが一杯になるたびに、割当量が次のピリオドを越えて持ち越されないようにする：

In [94]:
# Example 2
def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

ユーザが何かをしたいときは、常に最初に、使いたい量がバケツから得られるかどうかを確認する必要がある：

In [95]:
# Example 3
def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False  # Bucket hasn't been filled this period
    if bucket.quota - amount < 0:
        return False  # Bucket was filled, but not enough
    bucket.quota -= amount
    return True       # Bucket had enough, quota consumed


まずこのクラスを使うために、バケツに水を入れる。

In [96]:
# Example 4
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota=100)


次に、必要な割当量を求めることにする。

In [97]:
# Example 5
if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
print(bucket)

Had 99 quota
Bucket(quota=1)


最終的には、バケツの中にある水よりも多くの割当量を引き出そうとして、そこから進めなくなる。この場合、バケツの割当量は変更させない。

In [98]:
# Example 6
if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
print(bucket)

Not enough for 3 quota
Bucket(quota=1)


この実装の問題点は、割当量の初期値がわからないことである。割当量は時間とともに引き去られ、やがてゼロになる。その時点では、`deduct` は常に `False` を返すだろう。そうなったとき、 `deduct` の呼び出し元が割り当てられないのは、`Bucket` が割当量を引き出されたためなのか、それとも、`Bucket` にはそもそも最初から割当量がなかったのかを知ることは有用だろう。

この問題を解消するために、そのピリオドに与えた `max_quota` と使われた `quota_consumed` をクラスで記録しておくようにする。


In [99]:
# Example 7
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0

    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}, '
                f'quota_consumed={self.quota_consumed})')
# -- Example 8 --
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed

# -- Example 9 --
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Quota being filled during the period
            self.max_quota = amount + self.quota_consumed
        else:
            # Quota being consumed during the period
            self.quota_consumed = delta

元の `Bucket` クラスの前のインタフェースに合わせるため、`@property` メソッドを用いて、現在の割当量をその場で計算させる（上コード Example 8）

`quota` 属性への代入では、`fill` と `deduct` で使われているクラスの現在のインタフェースに合致するように特別な処理を行う（上コード Example 9）

上記のデモ用のコードを再度実行しても同じ結果となる。

In [100]:
# Example 10
bucket = NewBucket(60)
print('Initial', bucket)
fill(bucket, 100)
print('Filled', bucket)

if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')

print('Now', bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')

print('Still', bucket)

Initial NewBucket(max_quota=0, quota_consumed=0)
Filled NewBucket(max_quota=100, quota_consumed=0)
Had 99 quota
Now NewBucket(max_quota=100, quota_consumed=99)
Not enough for 3 quota
Still NewBucket(max_quota=100, quota_consumed=99)


一番良いところは、`Bucket.quota` を用いたコードを変更する必要も、クラスが変更されたことを知る必要もないことである。`Bucket` の新しい使い方では、`max_quota` や `quota_consumed` に直接アクセスしている。

`@property` の特に良い点は、時間をかけてより良いデータモデルへと逐次的に進めていけることである。この `Bucket` の例を読むと、「`fill` と `deduct` をそもそもインスタンスメソッドで実装すべきだった」と思ったかもしれない。 それはおそらく正しい（「項目37 組み込み型の深い入れ子にはせずクラスを作成する」参照）。しかし、実際の現場では、オブジェクトが適切ではないインタフェースで定義されていたり、ダメなデータコンテナとして振る舞っているところから始まる場合が多い。このようなことは、コードが時間とともに増大し、スコープが広がり、誰も長期にわたっての健全さを考慮せず複数のプログラマが関わるような場合に起こり得る。

`@property` は、実世界のコードで遭遇する問題を処理するのを助けるツールである。ただし、使いすぎないように気を付けるべきではある。繰り返し `@property` メソッドを拡張する羽目になったら、そのコードのひどい設計を何とか修正して使い続けるのはやめて、クラスをリファクタリングする時期であろう。


### 覚えておくこと

- `@property` を使って既存のインスタンス属性に新たな機能を追加する。
- `@property` を使って、より良いデータモデルへと逐次改善する。
- `@property` をあまりにも使いすぎると感じるようになったら、そのクラスとすべての呼び出し元をリファクタリングすることを考える。

## 項目46 再利用可能な@property メソッドにディスクリプタを使う

組み込みの@propertyにおける大きな問題は、 再利用です (「項目 44 get メソッドやset メソッ ドは使わず属性をそのまま使う」 や 「項目45 属性をリファクタリングする代わりに@property を考 える」参照)。 デコレートするメソッドを同じクラスの複数の属性で再利用することができないのです。
関連しないクラスでも再利用できません。

例えば、学生の宿題に付ける評価点は、0から100の範囲であることを確認したいとします。

In [101]:
# Example 1
class Homework:
    def __init__(self):
        self._grade = 0

    @property
    def grade(self):
        return self._grade

    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._grade = value

@property があれば、クラスは簡単に使えます。

In [102]:
# Example 2
galileo = Homework()
galileo.grade = 95
assert galileo.grade == 95

試験の点数も同様に確認したいとします。 試験には複数の科目があります。それぞれに評価を付けます。

In [103]:
# Example 3
class Exam:
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0

    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')

これではすぐに退屈になってしまいます。 試験の各科目に、新たな @property と関連する確認作業が必要です。


In [104]:
# Example 3
class Exam:
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0

    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')

# Example 4
    @property
    def writing_grade(self):
        return self._writing_grade

    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value

    @property
    def math_grade(self):
        return self._math_grade

    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value

galileo = Exam()
galileo.writing_grade = 85
galileo.math_grade = 99

assert galileo.writing_grade == 85
assert galileo.math_grade == 99

さらに、これでは汎用的ではありません。 この確認処理を宿題や試験というクラスの範囲を超えて 再利用するには、@propertyや_check_gradeメソッドを何度も繰り返し書く必要があります。

Python でこれをより良く行うには、ディスクリプタを使うことです。 ディスクリプタプロトコルは、属性アクセスを言語でどのように解釈するかを定義します。 ディスクリプタのクラスは、__get__メソッドや__set___ メソッドを提供して、点数の確認という作業の文をいちいち書かないで再利用する ことを可能にします。この目的のためには、ディスクリプタは、同じロジックをクラスの多くの異なる 属性で再利用できるという点で、 mix-in (「項目 41 Mix-in クラスで機能合成を考える」参照)よりも優れています、

Examという新たなクラスをGradeのインスタンスであるクラス属性を持たせるように定義します。
Gradeクラスは、ディスクリプタプロトコルを実装します。

In [105]:
# Example 5
class Grade:
    def __get__(self, instance, instance_type):
        pass

    def __set__(self, instance, value):
        pass

class Exam:
    # Class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

Gradeクラスがどのように働くかを説明する前に、 Exam インスタンスでこのようなディスクリプタ属性にアクセスしたとき、 Python が何を行うかを理解しておくことが重要でしょう。 属性値を代入します。

In [106]:
# Example 6
exam = Exam()
exam.writing_grade = 40

これは次のように解釈されます。

In [107]:
# Example 7
Exam.__dict__['writing_grade'].__set__(exam, 40)

属性値を取り出します。

In [108]:
# Example 8
exam.writing_grade


これは次のように解釈されます。

In [109]:
# Example 9
Exam.__dict__['writing_grade'].__get__(exam, Exam)

この振る舞いをさせているのは、オブジェクトの___getattribute__ メソッドです(「項目47 遅延属性には__getattr__, __getattribute__, __setattr__ を使う」 参照)。 手短に述べると、 Examインスタンスにwriting_grade という名前の属性がないと、 Python は、 Examクラスの属性を調べます。そのクラス属性が__get__や__set__ メソッドを持つオブジェクトなら、 Python は、ディスクリブタプロトコルを行いたいものと仮定します。

この振る舞いと Homeworkクラスにおける点数確認のために@propertyをどのように使ったかが
わかったので、 Gradeディスクリプタを使う最初の試みは次のようになります。


In [110]:
# Example 10
class Grade:
    def __init__(self):
        self._value = 0

    def __get__(self, instance, instance_type):
        return self._value

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._value = value

残念ながら、これには間違いがあり、 振る舞いの結果は期待通りではありませんでしたが、1つの
Exam インスタンスの複数の属性にアクセスすることは、期待通りに働きます。


In [111]:
# Example 11
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

Writing 82
Science 99


しかし、複数のExam インスタンスに対してこれらの属性をアクセスすると、予期せぬ振る舞いをします。


In [112]:
# Example 12
second_exam = Exam()
second_exam.writing_grade = 75
print(f'Second {second_exam.writing_grade} is right')
print(f'First  {first_exam.writing_grade} is wrong; '
      f'should be 82')

Second 75 is right
First  75 is wrong; should be 82


問題は、単一のGradeインスタンスが、すべてのExamインスタンスのクラス属性writing_grade として共有されているためです。 この属性のGrade インスタンスは、プログラムの生存期間で 一度だけ、 Examクラスが最初に定義されたときに構築され、 Examインスタンスが作られるたびに構
築されるのではありません。

この問題を解決するには、 Examインスタンスのそれぞれについて、 Grade クラスで、 その値を記録 保管する必要があります。 辞書にインスタンスごとに状態を保存することで解決します。


In [113]:
# Example 13
class Grade:
    def __init__(self):
        self._values = {}

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._values[instance] = value

この実装は単純できちんと働きますが、 まだ1つ、 メモリリークがあることを理解しておく必要があります。 辞書 _valuesはプログラムの生存期間を通じて、__set__に渡されるすべてのExamインスタンスへの参照を保持します。 こうなると、インスタンスの参照カウントはゼロになることがないので、 ガーベジコレクションでメモリ回収されなくなります (この種の問題をどう検出するかは 「項目81 メモリの使用とリークを理解するには tracemalloc を使う」 参照)。

かを説明する前に Exam

この問題を解くために、 Python の組み込みモジュール weakref を使います。このモジュールでは、_values に使う単純な辞書のために WeakKeyDictionary という特殊クラスを提供します。
WeakKeyDictionaryの振る舞いは特別なもので、実行時にExamインスタンスの最後に残っている プログラムでの参照が、 その辞書のキー集合で保持されているのだとわかったなら、そこからExamインスタンスを削除します。 Python は、この記録管理をしていて、すべてのExam インスタンスがもはや使われなくなったときには、_values 辞書が空になっていることを保証します。


In [114]:
# Example 14
from weakref import WeakKeyDictionary

class Grade:
    def __init__(self):
        self._values = WeakKeyDictionary()

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(
                'Grade must be between 0 and 100')
        self._values[instance] = value

Gradeディスクリプタをこのように実装すると、すべてが期待通りに働きます。

In [115]:
# Example 15
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75
print(f'First  {first_exam.writing_grade} is right')
print(f'Second {second_exam.writing_grade} is right')

First  82 is right
Second 75 is right


### 覚えておくこと

- ディスクリプタクラスを定義して、@property メソッドの振る舞いや確認作業を再利用す
る。
- WeakKeyDictionary を用いて、 ディスクリプタクラスがメモリリークを起こさないようにする。
- __getattribute__が属性のゲッターやセッターのためにディスクリプタプロトコルをどのように使っているか正確に理解して、予期せぬエラーを避ける。
